In [ ]:
# whisperのインストール　
#過去にインストール済みであれば実行不要
!pip install git+https://github.com/openai/whisper.git

In [ ]:
#過去にインストール済みであれば実行不要
!pip install pydub


In [ ]:
#過去にインストール済みであれば実行不要
!pip install openpyxl

In [ ]:
#過去にインストール済みであれば実行不要
pip install ffmpeg-python

In [1]:
#音声ファイルの分割の関数
#分割間隔は1分で指定しているが、任意で変更可能

import os
from pydub import AudioSegment
import pandas as pd

def split_audio_file(file_path, output_folder):
    # 入力ファイルの拡張子を取得
    file_extension = os.path.splitext(file_path)[1].lower()
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 音声形式に応じて読み込み方法を設定
    if file_extension == ".wav":
        audio = AudioSegment.from_wav(file_path)
    elif file_extension == ".mp3":
        audio = AudioSegment.from_mp3(file_path)
    elif file_extension == ".m4a":
        # .m4aファイルを読み込む
        audio = AudioSegment.from_file(file_path, format="m4a")
    elif file_extension == ".mp4":
        # .mp4ファイルを読み込む
        audio = AudioSegment.from_file(file_path, format="mp4")
    else:
        raise ValueError("サポートされていない音声形式です。")

    # 分割する時間間隔（１分）を取得
    split_interval = 1 * 60 * 1000  # ミリ秒単位

    # 分割した音声ファイルを保存するフォルダを作成
    os.makedirs(output_folder, exist_ok=True)

    # 音声ファイルを分割する
    for i, start_time in enumerate(range(0, len(audio), split_interval)):
        # 分割開始位置と終了位置を計算
        end_time = start_time + split_interval

        # 音声を分割
        split_audio = audio[start_time:end_time]

        # 出力ファイル名を作成
        output_file = os.path.join(output_folder, f"{file_name}_{i}{file_extension}")

        # 分割した音声ファイルを保存
        if file_extension == ".wav":
            split_audio.export(output_file, format="wav")
        elif file_extension == ".mp3":
            split_audio.export(output_file, format="mp3")
        elif file_extension == ".m4a":
            # Try encoding to MP4 (AAC) first, then rename to M4A
            temp_output_file = os.path.join(output_folder, f"{file_name}_{i}.mp4")
            split_audio.export(temp_output_file, format="mp4", codec="aac")
            output_file = os.path.join(output_folder, f"{file_name}_{i}.m4a")
            os.rename(temp_output_file, output_file)
        elif file_extension == ".mp4":
            split_audio.export(output_file, format="mp4")

        print(f"分割ファイル {output_file} を保存しました。")




In [2]:
#文字起こしの関数
#文字起こし精度は、tiny / base / small / medium / large から選択。
#大きいモデルほど精度が高く、処理時間が長くなる

import whisper
import os
import tkinter.filedialog
from pydub import AudioSegment
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import datetime
import ffmpeg


def transcribe_audio_files(input_folder, output_excel_file):
    # Whisperモデルをロード　
    # tiny / base / small / medium / large から精度を選択
    model = whisper.load_model("large")
    
    # フォルダ内の全音声ファイルをリストアップ
    file_list = [f for f in os.listdir(input_folder) if f.endswith('.m4a')]

    # ソートして順序を確保
    file_list.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # データフレームを作成
    df = pd.DataFrame(columns=['No', '音声ファイル', '変換結果'])

    # 各音声ファイルを文字変換する
    for i, file_name in enumerate(file_list):
        file_path = os.path.join(input_folder, file_name)
        print(f"変換開始: {file_path}")

        # 文字変換を実施
        result = model.transcribe(file_path)
        transcription = str(result["text"])
        print(f"変換結果: {transcription}")
        
        # 終了時間を記録
        end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"終了時間: {end_time}")

        # 結果をデータフレームに追加
        df.loc[i] = [i, file_path, transcription]

    # Excelへ書き出し
    workbook = Workbook()
    sheet = workbook.active

    # DataFrameの値をシートに書き込む
    for r in dataframe_to_rows(df, index=False, header=True):
        sheet.append(r)

    # ファイルへのリンクをセット
    for row in sheet.iter_rows(min_row=2, min_col=2, max_col=2):  # B列の値を処理
        cell = row[0]
        file_path = cell.value

        if file_path:
            cell.hyperlink = file_path
            cell.value = f'{file_path}'

    # Excelファイルを保存
    workbook.save(output_excel_file)
    print(f"変換結果を {output_excel_file} に保存しました。")

In [6]:
# 分割する音声のパス指定
file_path = "(音声ファイルのパス)"  # 元の音声ファイルのパス指定
output_folder = "(出力先ディレクトリのパス)"  # 分割したファイルの出力先指定

In [ ]:
# 音声ファイルの分割実行
split_files = split_audio_file(file_path, output_folder)

In [8]:
# 文字起こしのパス指定
input_folder = "(音声ファイルのパス)" #元の音声ファイルを格納したフォルダを指定
output_excel_file = "(出力先ディレクトリのパス、ファイル名の指定)" #出力先のExcelファイルを指定

In [ ]:
# 文字起こしの実行
transcribe_audio_files(input_folder, output_excel_file)